In [1]:
from pathlib import Path
import os
import win32com.client
from win32api import RGB
import shutil

print("Создание функций")
def delete_hidden_slides():
    i = 0
    # print(dirty_presentation.Slides.Count)
    while i <= dr_count_slides:
        slide = dirty_presentation.Slides[i]
        if slide.SlideShowTransition.Hidden == -1:
            slide.Delete()
        else:
            i += 1
def is_empty(data):
    return not data

def delete_empty_sections():
    for actual_section in range(1, dirty_presentation.SectionProperties.Count + 1):
        for section in range(1, dirty_presentation.SectionProperties.Count + 1):
            if is_empty(dirty_presentation.SectionProperties.SlidesCount(section)):
                dirty_presentation.SectionProperties.Delete(section, True)
                break

def delete_excess_shape(slide, shapes_not_del):
    shapes_not_del = [i.lower() for i in shapes_not_del]
    shapes_count = slide.Shapes.Count
    i = 1
    for _ in range(1, shapes_count+1):
        shape = slide.Shapes(i)
        shape_name = shape.name.lower()

        if shape_name not in shapes_not_del:
            shape.Delete() 
        else:
            i += 1
            

def create_header_file():
    shapes_count = template_presentation.Slides(1).Shapes.Count

    #Перебор всех объектов.
    not_delite = ['стрелка следующий слайд', 'стрелка предыдущий слайд', 'верхний фон', '..._-*-', '_...-*-']
    slide = template_presentation.Slides(1)
    delete_excess_shape(slide=slide, shapes_not_del=unique_names_array+not_delite)

    #измерить количество заголовком которое получится 
    bot_row = 40
    int_keff_sep_top = 5
    left_long_all_figure = 0
    count_head_slide = 1

    for unique_name in unique_names_array:
        for i in range(1, shapes_count+1):
            shape = slide.Shapes(i)
            shape_name = shape.name.lower()
            if shape_name==unique_name: 
                long_shape = shape.Width 
                
                left_long_all_figure += long_shape
                left_long_all_figure += int_keff_sep_top

                if left_long_all_figure > 890:  #Left=890 стрелки
                    count_head_slide += 1
                    left_long_all_figure = slide.Shapes('..._-*-').Width + slide.Shapes('_...-*-').Width
    #slide copy
    if count_head_slide != 1:
        template_presentation.Slides(1).Copy()
        for n_sl in range(1, count_head_slide):
            template_presentation.Slides.Paste(Index=1+n_sl)


    #если первый слайд 
    # берём все фигуры до > 890 + slide.Shapes('..._-*-')
    # если ниже добавляем последний элемент эту фигуру
    # записываем на каком названиее в переборе листа мы остановились. После его будем брать по индексу. 

    #если слайд последний
    # первая фигура slide.Shapes('_...-*-')
    # после проставляем оставшиеся 

    # без условий (не первый не последний)
    # первый слайд проставляем фигуру slide.Shapes('_...-*-')
    # проставляем фигуры между. 
    # берём все фигуры до > 890 + slide.Shapes('..._-*-')

    # возможно это удобнее делать в местое где я буду проставлять гиппер ссылки
    # т.к. я не знаю как идентифицировать первую фигуру 
    # есть вариант создать список из списков названий. И после подтягивать каждое незвание по имени.  

    # Проставляю заголовки в требуемом порядке. С определённым растоянием между ними.
    int_keff_sep = 2
    top_row = 5
    #Если только один слайд
    if count_head_slide == 1:
        left_long = int_keff_sep # = 2
        # shapes_count = template_presentation.Slides(1).Shapes.Count
        slide = template_presentation.Slides(1)
        # top_row = int_keff_sep_top
        slide_list_heads = []
        for unique_name in unique_names_array:
            # for i in range(1, shapes_count+1):
            shape = slide.Shapes(unique_name)
            shape_name = shape.name.lower()
            if shape_name==unique_name: 
                long_shape = shape.Width 

                shape.Top  = top_row
                shape.left = left_long
                
                left_long = left_long + long_shape
                left_long = left_long + int_keff_sep
                
                slide_list_heads.append(shape_name)

        not_delite_1 = not_delite[:]
        not_delite_1.remove('..._-*-')
        not_delite_1.remove('_...-*-')

        delete_excess_shape(slide=slide, shapes_not_del=not_delite_1+slide_list_heads)

    #если слайдов заголовок больше             
    if count_head_slide > 1:
        slides_list_list_heads = []
        print('count_head_slide', count_head_slide)
        for n_slide in range(1, count_head_slide+1):
            print('n_slide', n_slide)

            # первый слайд
            if n_slide == 1:
                slide = template_presentation.Slides(n_slide)
                
                left_long = int_keff_sep # = 2
                shapes_count = template_presentation.Slides(1).Shapes.Count
                
                slide_list_heads = []
                for unique_name in unique_names_array:
                    shape = slide.Shapes(unique_name)
                    shape_name = shape.name.lower()
                    print(left_long, slide.Shapes('_...-*-').Width, shape.Width, int_keff_sep)
                    if left_long >= 890 - slide.Shapes('_...-*-').Width - shape.Width - int_keff_sep:
                        shape = slide.Shapes('_...-*-')
                        shape.Top = top_row
                        shape.left = left_long
                        break
                    if shape_name==unique_name: 
                        long_shape = shape.Width 

                        shape.Top  = top_row
                        shape.left = left_long
                        
                        left_long = left_long + long_shape
                        left_long = left_long + int_keff_sep

                        slide_list_heads.append(shape_name)
                #удаляем лишнее
                not_delite_1 = not_delite[:]
                not_delite_1.remove('..._-*-')
                delete_excess_shape(slide=slide, shapes_not_del=not_delite_1+slide_list_heads)        
                slides_list_list_heads.append(slide_list_heads)
            #последний слайд
            elif n_slide == count_head_slide:
                slide = template_presentation.Slides(n_slide)
                
                left_long = int_keff_sep # = 2
                slide_list_heads = []

                used_head = [ii for i in slides_list_list_heads for ii in i]
                not_used_head = [i for i in unique_names_array if i not in used_head]

                shape     = slide.Shapes('..._-*-')
                shape.Top = top_row
                shape.left= left_long
                left_long += long_shape
                left_long += int_keff_sep

                print(n_slide, 'n_slide', used_head, 'used_head', not_used_head, 'not_used_head')
                for unique_name in not_used_head:
                    shape = slide.Shapes(unique_name)
                    shape_name = shape.name.lower()
                    print(left_long, slide.Shapes('_...-*-').Width, shape.Width, int_keff_sep)

                    if shape_name==unique_name:
                        long_shape = shape.Width 

                        shape.Top  = top_row
                        shape.left = left_long
                        left_long = left_long + long_shape
                        left_long = left_long + int_keff_sep

                        slide_list_heads.append(shape_name)
                # удаление ненужных
                not_delite_1 = not_delite[:]
                not_delite_1.remove('_...-*-')
                delete_excess_shape(slide=slide, shapes_not_del=not_delite_1+slide_list_heads)        
                slides_list_list_heads.append(slide_list_heads)
            #Слайды по середине
            else:
                slide = template_presentation.Slides(n_slide)
                left_long = int_keff_sep # = 2
                slide_list_heads = []

                used_head = [ii for i in slides_list_list_heads for ii in i]
                not_used_head = [i for i in unique_names_array if i not in used_head]

                shape     = slide.Shapes('..._-*-')
                shape.Top = top_row
                shape.left= left_long
                left_long += long_shape
                left_long += int_keff_sep

                for unique_name in not_used_head:
                    shape = slide.Shapes(unique_name)
                    shape_name = shape.name.lower()
                    print(left_long, slide.Shapes('_...-*-').Width, shape.Width, int_keff_sep)
                    if left_long >= 890 - slide.Shapes('_...-*-').Width - shape.Width - int_keff_sep:
                        shape = slide.Shapes('_...-*-')
                        shape.Top = top_row
                        shape.left = left_long
                        break
                    if shape_name==unique_name: 
                        long_shape = shape.Width 

                        shape.Top  = top_row
                        shape.left = left_long
                        
                        left_long = left_long + long_shape
                        left_long = left_long + int_keff_sep

                        slide_list_heads.append(shape_name)
                #удаляем лишнее
                not_delite_1 = not_delite[:]
                delete_excess_shape(slide=slide, shapes_not_del=not_delite_1+slide_list_heads)        
                slides_list_list_heads.append(slide_list_heads)


    return slides_list_list_heads

def get_hyperlink_next_early(slide, names_array, n_sl, n_sl_max, heads_lists):
    if n_sl == 1 and n_sl_max == 1:
        return None #если слайд одни нет фигур  = ..._  и _...
    elif n_sl == 1 and n_sl_max != 1:
        shape = slide.Shapes('_...-*-')
        index_next_slide = n_sl
        first_head_next_slide = heads_lists[index_next_slide][0]
        n_slide_this_head = names_array.index(first_head_next_slide) + 1
        shape.ActionSettings(1).Hyperlink.Address = '#' + str(n_slide_this_head)
    elif n_sl == n_sl_max:
        shape = slide.Shapes('..._-*-')
        index_early_slide = n_sl - 2
        first_head_next_slide = heads_lists[index_early_slide][-1]
        n_slide_this_head = names_array.index(first_head_next_slide) + 1
        shape.ActionSettings(1).Hyperlink.Address = '#' + str(n_slide_this_head)
    else:
        shape = slide.Shapes('_...-*-')
        index_next_slide = n_sl
        first_head_next_slide = heads_lists[index_next_slide][0]
        n_slide_this_head = names_array.index(first_head_next_slide) + 1
        shape.ActionSettings(1).Hyperlink.Address = '#' + str(n_slide_this_head)

        shape = slide.Shapes('..._-*-')
        index_early_slide = n_sl - 2
        first_head_next_slide = heads_lists[index_early_slide][-1]
        n_slide_this_head = names_array.index(first_head_next_slide) + 1
        shape.ActionSettings(1).Hyperlink.Address = '#' + str(n_slide_this_head)


def get_hyperlink(slide, names_array, n_sl, n_sl_max, heads_lists): #TODO
    n_slide_index = n_sl - 1
    head_list = heads_lists[n_slide_index]
    
    for sh_name in head_list:
        shape = slide.Shapes(sh_name)
        n_slide_this_head = names_array.index(sh_name) + 1
        shape.ActionSettings(1).Hyperlink.Address = '#' + str(n_slide_this_head)

    get_hyperlink_next_early(slide, names_array, n_sl, n_sl_max, heads_lists)
                    

def headder_delite(del_head_on_top, del_head_on_templ_sh, templ_shapes_all):
    """Удаляет или объекты котоыре выше 15 без разбора названия или объекты в templ_shapes_all"""
    dr_count_slides=dirty_presentation.Slides.Count
    for i in range(1, dr_count_slides+1):
        slide = dirty_presentation.Slides(i)
        ii = 1
        for _ in range(1, slide.Shapes.Count+1):
            shape = slide.Shapes(ii)
            shape_name = shape.name.lower()
#             print(shape_name)

            if del_head_on_top == True and shape.Top < 35 and 'заголовок' not in shape_name and 'title' not in shape_name:
#                 print('Удаление 2')
                slide.Shapes(ii).Delete()
                #удалюя если объект выше 35 и "не заголовок" 
            elif del_head_on_templ_sh == True and  shape_name in templ_shapes_all:
#                 print('Удаление 1')
                slide.Shapes(ii).Delete()
            else:
                ii = ii + 1
def shape_head_white_color(slide):
    title_ = slide.Shapes.Title.TextEffect.text.lower()
    shape_count = slide.shapes.Count
    print(title_)
    for n_sh in range(1, shape_count+1):
        shape_white = slide.shapes(n_sh)
        shape_white_name = shape_white.name.lower()
        print(shape_white_name, n_sh, 'shape_white_name')
        if shape_white_name == title_:
            shape_white.Fill.ForeColor.RGB = RGB(255, 255, 255)
            break

            
def load_head_slide(names_array, slides_list_list_heads):
    dr_count_slides=dirty_presentation.Slides.Count
    templ_count = template_presentation.Slides.Count
    #копируем все слайды
    for i in range(1, templ_count+1): 
        template_presentation.Slides(i).Copy()
        dirty_presentation.Slides.Paste(Index=dr_count_slides+i)
    
    #ПРОСТАВЛЕНИЕ ССЫЛОК
    for i in range(1, templ_count+1):
        slide = dirty_presentation.Slides(dr_count_slides+i)
        get_hyperlink(slide=slide, names_array=names_array, n_sl=i, n_sl_max=templ_count, heads_lists=slides_list_list_heads)


    #Проставляю красивые заголовки.
    n_slide_copy_before = 0
    for n_sl in range(1, dr_count_slides+1):
        slide = dirty_presentation.Slides(n_sl)
        title_slide = names_array[n_sl-1]
        
        #не эффективный перебор всех названий слайда
        flag = False
        for n_head, list_head in enumerate(slides_list_list_heads):
            for head in list_head:
                if title_slide == head:
                    n_slide_copy = dr_count_slides + n_head + 1
                    flag = True
                    break
                if flag:
                    break
        
        # print(n_slide_copy, n_sl)
        if n_slide_copy != n_slide_copy_before:
            print('Копировал', n_slide_copy)
            dirty_presentation.Slides(n_slide_copy).Shapes.Range().Copy()
        dirty_presentation.Slides(n_sl).Shapes.Paste().ZOrder(1)

        n_slide_copy_before = n_slide_copy
        #ссылка стрелки
        slide_arrow = dirty_presentation.Slides(n_sl)
        count_shape_arrow = slide_arrow.Shapes.Count
        
        for n_sh in range(1, count_shape_arrow+1):
            shape = slide_arrow.Shapes(n_sh)
            shape_name = shape.name.lower()
            
            if n_sl != 1:
                if shape_name == 'стрелка предыдущий слайд':
                    shape.ActionSettings(1).Hyperlink.Address = '#' + str(n_sl - 1)
            if n_sl != dr_count_slides:
                if shape_name == 'стрелка следующий слайд':
                    shape.ActionSettings(1).Hyperlink.Address = '#' + str(n_sl + 1)
    #Подкрашиваю
    for n_sl in range(1, dr_count_slides+1):
        slide = dirty_presentation.Slides(n_sl)
        shape_head_white_color(slide)
    
    
    #Удаляю доп листы с заголовками
    ii = dr_count_slides + 1
    for i in range(dr_count_slides + 1, dirty_presentation.Slides.Count+1):
        dirty_presentation.Slides(ii).Delete()
    
        
def f_push_shape_on_bot(bot_row):
    dr_count_slides=dirty_presentation.Slides.Count
    
    for n_sl in range(1, 1 + dr_count_slides):
        slide = dirty_presentation.Slides(n_sl)
        count_shapes = slide.Shapes.Count

        for s_sh in range(1, 1+count_shapes):
            shape = slide.Shapes(s_sh)
            shape_name = shape.name.lower()

            if 'заголовок' not in shape_name:
                shape_top = shape.Top
                shape.Top = shape_top + bot_row #40 высота шапки страндартная.
    
            
def reduse_all_shape(kef_low):
    dr_count_slides=dirty_presentation.Slides.Count
    for n_sl in range(1, 1 + dr_count_slides):
        slide = dirty_presentation.Slides(n_sl)
        count_shapes = slide.Shapes.Count

        for s_sh in range(1, 1+count_shapes):
            shape = slide.Shapes(s_sh)

            if '-*-' in shape.name: #не тротаем базовые фигуры 
                continue

            shape_width_old = shape.Width
            shape_Height_old = shape.Height
            shape_top_old = shape.Top
            # new
            shape.Width = shape_width_old * kef_low
            shape.Height = shape_Height_old * kef_low
            shape.Top = shape_top_old * kef_low

            if shape.HasTextFrame:
                shape_text_size = shape.TextFrame.TextRange.Font.Size
                shape.TextFrame.TextRange.Font.Size = int(shape_text_size * kef_low * 0.90)
                #домножаем на 0.9 т.к. тектс обынчо не помещается в новую рамку не смотря на уменьшение.


def main(path_tipl, path_file, del_head_on_top, del_head_on_templ_sh, reduse_shapes, push_shape_on_bot):
    global dirty_presentation
    global template_presentation

    file      = Path(path_file) 
    file_copy = file.parent.joinpath(file.stem + '_copy' + file.suffix)
    file_itog = file.parent.joinpath(file.stem + '_itog' + file.suffix)
    file_itog_pdf = file.parent.joinpath(file.stem + '_itog_pdf' + ".pdf")
    template      = Path(path_tipl)
    template_copy = template.parent.joinpath(template.stem + '_copy' + template.suffix)
    template_itog = template.parent.joinpath(template.stem + '_itog' + template.suffix)



    shutil.copy(file, file_copy)
    shutil.copy(template, template_copy)

    pptx = win32com.client.Dispatch("PowerPoint.Application")
    dirty_presentation    = pptx.Presentations.Open(str(file_copy), ReadOnly=True, WithWindow=False)
    template_presentation = pptx.Presentations.Open(str(template_copy), ReadOnly=True, WithWindow=False)

    global names_array
    global dr_count_slides
    global unique_names_array

    dr_count_slides = dirty_presentation.Slides.Count
    
    delete_hidden_slides()
    delete_empty_sections()
    


    
    names_array = []
    for i in range(1, dirty_presentation.Slides.Count+1):
        name_title = dirty_presentation.Slides(i).Shapes.Title.TextEffect.text.lower().strip()
        names_array.append(name_title)
    print(len(names_array), dr_count_slides)
    unique_names_array = []
    for name in names_array:
        if name not in unique_names_array:
            unique_names_array.append(name)


    templ_shapes_all = []
    tem_slide =  template_presentation.Slides(1)
    for i in range(1, tem_slide.Shapes.Count+1):
        shape_name = tem_slide.Shapes(i).name.lower()
        templ_shapes_all.append(shape_name)

    slides_list_list_heads = create_header_file()
    template_presentation.SaveAs(template_itog)
    bot_row = 40

    print(dirty_presentation.PageSetup.SlideHeight, 'SlideHeight')
    print(dirty_presentation.PageSetup.SlideWidth, 'SlideWidth')
    
    dir_pres_height = dirty_presentation.PageSetup.SlideHeight
    kef_low = (dir_pres_height - bot_row) / dir_pres_height


    if reduse_shapes == True :
        reduse_all_shape(kef_low)
        print('Фигуры уменьшены')

    if push_shape_on_bot == True:
        f_push_shape_on_bot(bot_row)
        print('Фигуры сдвинуты вниз')

    template_presentation.SaveAs(template_itog)

    headder_delite(del_head_on_top, del_head_on_templ_sh, templ_shapes_all)



    load_head_slide(names_array, slides_list_list_heads)
    dirty_presentation.SaveAs(file_itog)
    dirty_presentation.SaveAs(file_itog_pdf, 32)
    
    #удаляю копии с которыми работал
    os.remove(file_copy)
    os.remove(template_copy)
    
    template_presentation = None
    dirty_presentation = None
    pptx = None
    #в место
#     dirty_presentation.Close()
#     template_presentation.Close()
#     pptx.Quit()
    print('Выполнено!')
    print('Можно закрывать')


import PySimpleGUI as sg
from pathlib import Path
import os
import os
import win32com.client
from win32api import RGB
import shutil





########################## Визульная часть
try:
    with open('save_path_tipl.txt') as d_s:
        path_tipl = d_s.read()
except:
    path_tipl = ''

try:
    with open('save_path_file.txt') as d_s:
        path_file = d_s.read()
except:
    path_file = ''

layout = [[sg.Input(default_text=path_tipl, key='-path_tipl_new-'), sg.FileBrowse(button_text='Выбрать шаблон', key='-path_tipl_new-'), sg.Button('сохранить путь',enable_events=True, key='-save_path_tipl-')],
          [sg.Input(default_text=path_file, key='-path_file_new-'), sg.FileBrowse(button_text='Выбрать презентацию', key='-path_file_new-'), sg.Button('сохранить путь',enable_events=True, key='-save_path_file_-')],
#           [sg.Text('После сохранения следующий запуск будет с сохраненными параметрами')],
#           [sg.Checkbox('удалать старую шапку по названию',key='-del_head_on_templ_sh-', default=True)],
          [sg.Text('При запуске:')],
          [sg.Text('* удаляется старая шапка по названиям из шаблона')],
          [sg.Text('* удаляются скрытые слайды')],
          [sg.Text('* формируется новая шапка')],
          [sg.Checkbox('Сдвинуть всё вниз на величину шапки',key='-push_shape_on_bot-', default=False)],
          [sg.Checkbox('Уменьшить всё, чтобы помещалось после сдвига вниз',key='-reduse_shapes-', default=False)],
          [sg.Checkbox('удалать старую шапку по высоте объектов(может удалить не только заголовок)',key='-del_head_on_top-', default=False)],
          [sg.Button('Формировать',enable_events=True, key='-FUNCTION-', font='Helvetica 16')],
          [sg.Text('Итоговый файл будет сохранён в папку с исходным файлом', size=(60, 3), key='-text-', font='Helvetica 10')],
          [sg.Output(size=(100,10))]
         ]
# reduse_shapes
window = sg.Window('Добавление заголовков презентации по шаблону', layout)


flag_do = True
while True:
    event, values = window.read()
    
    if event in (sg.WIN_CLOSED, 'Exit'):
        break
    path_tipl_new = values['-path_tipl_new-']
    path_file_new =  values['-path_file_new-']  
#     del_head_on_templ_sh = values['-del_head_on_templ_sh-']
    del_head_on_top =  values['-del_head_on_top-']
    push_shape_on_bot = values['-push_shape_on_bot-']
    reduse_shapes = values['-reduse_shapes-']
    
    del_head_on_templ_sh = True
    print(push_shape_on_bot, push_shape_on_bot)
    print(type(push_shape_on_bot))
    

    path_true = False

    #Файл шаблон
    if path_tipl_new != '':
        path_tipl = path_tipl_new
    if event == '-save_path_tipl-':
        with open('save_path_tipl.txt', 'w') as f:
            f.write(path_tipl_new)
            print(path_tipl_new, 'сохранён')
            
    #Файл рабочий 
    if path_file_new != '':
        path_file = path_file_new
    if event == '-save_path_file_-':
        with open('save_path_file.txt', 'w') as f:
            f.write(path_file_new)
            print(path_file_new, 'сохранён')

    if path_tipl != '' and  path_file != '':
        path_true = True

    if event == '-FUNCTION-' and path_true is True and flag_do == True:
        flag_do = False #для отсутствия возможности перезапустить Иначе всё повиснет. 
        print(Path(path_file))
        print(Path(path_tipl))
        print('Вызов функции')
        # print(type(del_head_on_top), del_head_on_templ_sh)
        main(path_tipl, path_file, del_head_on_top, del_head_on_templ_sh, reduse_shapes=reduse_shapes, push_shape_on_bot=push_shape_on_bot)
    elif event == '-FUNCTION-':
        print(f"""Нет запуска 
              path_tipl = {path_tipl}
              path_file = {path_file}
              flag_do   = {flag_do}"""
              )
window.close()
